# Decision Trees

### High Level Summary
A decision performs a series of binary splits at various nodes of a binary tree. Starting with a single top node, the "best split" is performed on the data and its features. Best split, generally, means the split that provides the greatest information gain among all possible splits. The information gain is computed via an impurity function, which is a parameter of the decision tree (i.e. user specifies which impurity function to use).

## Algorithm
TODO
### Terminal node prediction refinement
TODO

## Spark MLlib Implementation

Spark's MLlib implementation is optimized for parallel computation. A single decision tree is actually trained using the Random Forest training algorithm where the forest size equals one. Before the trees are trained, a function  is called to find all possible splits for the features of the dataset. Continuous and categorical features are treated differently so that the number of splits is manageable. The data is then converted to a tree point representation where each feature vector is replaced with a vector of bin or split indices that indicate which split that data point falls into for each feature. The algorithm then places top nodes in a queue (for decision tree there is only one top node) and begins to train. 

The training is done by finding the best possible split, given the impurity function, for each node. In each partition, the "sufficient statistics" are computed for the data points in that partition for each possible split. The data is then combined for each partition on the driver node, which then selects the best split. "Sufficient statistics" are the statistics about the labels of the data in each bucket which are sufficient to compute the impurity and the prediction. For a Variance impurity, the split is based on sum of squares, so sum(y^2), sum(y), and count(y) are all collected for each partition. These can be combined by addition across the partitions and the statistics available at the driver node are sufficient for computing the best split. Lather, rinse, repeat.